## This file will runs experiments on ImageNet Dataset

- this file will 
   - find segmentation hyperparamers for all 150 images 
   - generate expalantions for all the images in data folder and will save the results as csv file 
<br><br>
- you may run the full benchmark on all the 150 images which will take more than one day to run, or run a simplified version on a single image which will take less time.

This code was tested on two windows laptop and a mac-os laptop with following specifications:

```
3.1. Dell G5      (Corei7 12 CPUs 2.20Ghz, 16GB RAM, NVidia 1060 GPU 6GB)
3.2. Santech XN2  (Corei9 20 CPUs 2.60Ghz, 16GB RAM, NVidia 4070 GPU 8GB)
3.3. Macbook Pro  (M1, 16GB RAM)
```

### Import Libraries

In [ ]:
import os
import importlib
import utils as ut
from datetime import date,datetime
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))    # Stretch Notebook Width to 98% size of the Screen
# It will use the modified code of lime downloaded from https://anonymous.4open.science/r/lime-stratified-BDE5/
from tensorflow.keras.applications.resnet50 import preprocess_input
from lime_stratified.lime import lime_image

### Setting Path

In [ ]:
# Get Current Working Directory and joining subfolders and subfiles path
Main_dir =   os.getcwd()
DS_path =  os.path.join(Main_dir, "data")
result_folder = os.path.join(Main_dir, "result")
json_file    =  os.path.join(DS_path,"imagenet_class_index.json")

## BlackBox Model

Load BlackBox Model, here ResNet50 Model is loaded

In [ ]:
def bb_predict(imgs):
    # On some platform, you will need model.predict(..) instead of model(..)
    return model.predict(preprocess_input(imgs), verbose=False)
#     return model(preprocess_input(imgs))

In [ ]:
model_name = 'ResNet50'
model = ut.load_model(model_name)
# getting ImageNet class names
class_names = ut.get_ImageNet_ClassLabels(json_file) 

In [ ]:
now = datetime.now()
print("Example Started\t\t:\t\t\t", now.strftime("%d/%m/%Y %H:%M:%S"))

# Hyperparameters

In [ ]:
#######################################################################################################
## SEGMENTATION PARAMETERS
seg_algo                    =  'quickshift'
segs_list                   =  [50,100,150,200]           # Target No of Segments in Image Explanation
# These hyperparameters can be used to create LIME Image Explanations
hide_color                  =  [None]
use_stratification          =  [False,True]
num_samples                 =  1000    # samples in the synthetic neighbourhood needed to compute every explanation
batch_size                  =  500     # batch size to compute the synthetic neighbourhood 
# Top Labels: Upto how much labels the explanation is needed, repeat_exp: Upto how much Explanation is needed to be 
# to get an average explanation to remove randomness 
top_labels                  =  2      # labels for the   
repeat_exp                  =  10     # aeverage of 10 explanations for the same image to reduce randomness 
#  Results of Experiments
plot_prediction             =  True #  Set it to True if plots for explanations are needed to be plot, Default: False
plot_segments               =  True
plot_explanation            =  True
plot_classification_score   =  True
plot_heatmap                =  True
plot_image_mask             =  True 
save_explanations_as_plot   =  True #  Set it to True if plots for explanations are needed to be saved also, Default: False

###################    PATHS for Saving results      ##########################
sub_results__ = os.path.join(result_folder,str(segs_list))
sub_results_ = os.path.join(sub_results__,str(num_samples))
_ = ut.check_folders(sub_results__)
############################### 
create_segments_file        =  True ### IF SEGMENT FILE Needs to be regenerated, SET 'compute_segments' as True
create_data_file            =  True  ### IF SEGMENT FILE IS LOADED AND COMPUTE EXPERIMENTS ARE REQUIRED, SET 'compute_experiments' as True

### Full Benchmark
Compute the benchmark by using these pararmeters on the 150 images 

In [ ]:
# files                       =  range(0,150,1)

### Simplified Benchmark

Use these pararmeters on the single on a image 

In [ ]:
files                       =  range(124,125,1)

### Create Segments 0-100, 100-200, 200-300
Identify the max_dist hyperparamer needed to get exactly the desired amount of segments in of images

In [ ]:
df_seg = ut.segmentation_module(create_segments_file,files,DS_path,result_folder,model,segs_list,seg_algo)

In [ ]:
df_seg

### Run the Experiments and Save the CSV File


In [ ]:
df_data = ut.explanation_module(create_data_file,files,df_seg,DS_path,sub_results_,result_folder,segs_list,
                                 model,model_name,class_names,save_explanations_as_plot,use_stratification,
                                 plot_prediction,plot_segments,plot_heatmap,plot_image_mask,hide_color,
                                 num_samples,repeat_exp,top_labels,batch_size,lime_image,bb_predict)

In [ ]:
df_data.head()

In [ ]:
now = datetime.now()
print("Example Started\t\t:\t\t\t", now.strftime("%d/%m/%Y %H:%M:%S"))